In [1]:
!ls data

anomaly_label.csv  Event_traces.csv


In [2]:
import numpy as np

trace_file = "data/Event_traces.csv"
label_file = "data/anomaly_label.csv"

SOS = "<SOS>"
EOS = "<EOS>"

background_label = "Normal"

In [3]:
label_dict = {"Normal": 0, "Anomaly": 1}
events_set = set()

events = list()
labels = list()

for i, (data_line, label_line) in enumerate(zip(open(trace_file, "rt"), open(label_file, "rt"))):
    if i==0:
        continue
    event_trace = data_line.split("[")[1].strip("\"").strip(",").strip("\"").strip("]").split(",")
    for event in event_trace:
        events_set.add(event)
    events.append(event_trace)
    
    label = label_line.split(",")[-1].strip("\n")
    labels.append(label_dict[label])
events_set.add(SOS)
events_set.add(EOS)

In [4]:
event_dict = {event: i for i, event in enumerate(events_set)}
alphabet_size = len(event_dict)

events_oh = list()
for i in range(len(events)):
    trace = events[i]
    trace_oh = np.zeros((len(trace)+1, alphabet_size))
    for j, event in enumerate(trace):
        events[i][j] = event_dict[event]
        trace_oh[j][event_dict[event]] = 1
    events[i].insert(0, event_dict[SOS])
    trace_oh[-1][event_dict[EOS]] = 1
    events_oh.append(trace_oh)
events[5:8], events_oh[5:8]

([[10,
   12,
   12,
   13,
   12,
   1,
   5,
   1,
   5,
   1,
   5,
   2,
   2,
   2,
   4,
   4,
   4,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   26,
   19,
   19,
   19,
   19,
   19,
   8,
   8,
   8,
   6,
   6,
   6],
  [10,
   12,
   13,
   12,
   12,
   1,
   1,
   5,
   5,
   1,
   5,
   2,
   2,
   2,
   4,
   4,
   4,
   19,
   19,
   19,
   19,
   19,
   19,
   26,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   8,
   8,
   8,
   6,
   6,
   6],
  [10,
   12,
   12,
   12,
   13,
   1,
   5,
   1,
   5,
   2,
   2,
   1,
   5,
   2,
   4,
   4,
   4,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   26,
   19,
   19,
   8,
   8,
   8,
   6,
   6,
   6]],
 [array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0.,

In [5]:
event_dict

{'E10': 0,
 'E11': 1,
 'E26': 2,
 'E18': 3,
 'E2': 4,
 'E9': 5,
 'E21': 6,
 'E15': 7,
 'E23': 8,
 '<EOS>': 9,
 '<SOS>': 10,
 'E28': 11,
 'E5': 12,
 'E22': 13,
 'E12': 14,
 'E14': 15,
 'E19': 16,
 'E20': 17,
 'E16': 18,
 'E4': 19,
 'E8': 20,
 'E24': 21,
 'E29': 22,
 'E17': 23,
 'E13': 24,
 'E6': 25,
 'E3': 26,
 'E25': 27,
 'E7': 28,
 'E1': 29,
 'E27': 30}

In [6]:
len(events), np.unique(np.array(labels),return_counts=True)

(575061, (array([0, 1]), array([558223,  16838])))

In [7]:
_, (n_normal, n_anomalies) = np.unique(np.array(labels),return_counts=True)
n_normal, n_anomalies

(558223, 16838)

In [8]:
import random

X_train = list()
X_train_oh = list()
X_test = list()
X_test_oh = list()
y_test = list()

n_normal_test = n_anomalies # we want test set to be balanced in labels
p_test = n_normal_test / (n_normal - n_anomalies)
for event, event_oh, label in zip(events, events_oh, labels):
    p = random.uniform(0, 1)

    if label==1 or p <= p_test:
        X_test.append(event)
        X_test_oh.append(event_oh)
        y_test.append(label)
    else:        
        X_train.append(event)
        X_train_oh.append(event_oh)

len(X_train), len(X_test), np.unique(y_test, return_counts=True)

(540942, 34119, (array([0, 1]), array([17281, 16838])))

In [9]:
import pickle as pk

pk.dump({
    "X": X_train,
    "X_oh": X_train_oh,
    "alphabet_size": alphabet_size,
    "event_mapping": event_dict,
    },
    open("train_data.pk", "wb")   
)
pk.dump({
    "X": X_test,
    "X_oh": X_test_oh,
    "y": y_test,
    "alphabet_size": alphabet_size,
    "event_mapping": event_dict,
    },
    open("test_data.pk", "wb")   
)

In [10]:
!ls

data	  preprocess_data.ipynb  test_model.ipynb  train_model.ipynb
model.h5  test_data.pk		 train_data.pk
